In [1]:
import pandas as pd

def analisar_reembolsos(arquivo_csv):
    """
    Analisa dados de reembolso para identificar padrões de comportamento suspeitos de transportadoras e motoristas.

    Args:
        arquivo_csv (str): O caminho para o arquivo CSV contendo os dados de reembolso.

    Returns:
        DataFrame: DataFrame original com as colunas de risco e regras adicionadas.
    """

    try:
        df = pd.read_csv(arquivo_csv)

        # Converter colunas de data para datetime
        date_columns = ['Data da criação do reembolso', 'Data da ultima atualização do reembolso', 'Data da aprovação ou rejeição do reembolso']
        for col in date_columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

        # Converter ID do motorista para int (se for seguro fazer isso)
        df['ID do motorista'] = df['ID do motorista'].fillna(0).astype(int)
        df['Ajudante (1) ou Motorista (0)'] = df['Ajudante (1) ou Motorista (0)'].fillna(0).astype(int)

    except FileNotFoundError:
        print(f"Erro: Arquivo '{arquivo_csv}' não encontrado.")
        return None
    except Exception as e:
        print(f"Erro ao ler o arquivo: {e}")
        return None

    # Funções auxiliares para as regras (INDIVIDUAIS)
    def regra_valor_contrato(row):
        valor_contrato = row['Valor do contrato']
        valor_reembolso = row['Valor total do reembolso']
        if valor_contrato >= 3000000 and valor_reembolso >= 0.5 * valor_contrato:
            return True
        return False

    def regra_valor_alto(row):
        if row['Valor total do reembolso'] > 3000000:
            return True
        return False

    def regra_tempo_aprovacao(row):
        if pd.notna(row['Data da aprovação ou rejeição do reembolso']) and pd.notna(row['Data da criação do reembolso']):
          tempo_aprovacao = (row['Data da aprovação ou rejeição do reembolso'] - row['Data da criação do reembolso']).total_seconds() / 60
          if tempo_aprovacao < 1 and row['Valor total do reembolso'] > 500:
              return True
        return False

    def regra_horario(row):
        hora_transacao = row['Data da criação do reembolso'].hour if pd.notna(row['Data da criação do reembolso']) else -1
        if hora_transacao >= 21 and row['Valor total do reembolso'] > 300:
            return True
        return False

    # Aplica as regras linha a linha (INDIVIDUAIS)
    df['reembolso_suspeito_individual'] = df.apply(lambda row: any([
        regra_valor_contrato(row),
        regra_valor_alto(row),
        regra_tempo_aprovacao(row),
        regra_horario(row)
    ]), axis=1)

    #Correção da escala de unidades
    df['Data da criação do reembolso Date'] = df['Data da criação do reembolso'].dt.date #Cria coluna só com a data

    # Agrupa por data, transportadora e motorista
    df_diario = df.groupby(['Data da criação do reembolso Date', 'Nome da transportadora', 'Nome do motorista', 'Valor do contrato', 'Número de dias agenciados do contrato']).agg( #type: ignore
        total_reembolsos=('Valor total do reembolso', 'sum'),
        num_reembolsos=('Valor total do reembolso', 'count'),
        num_reembolsos_alto_valor = ('Valor total do reembolso', lambda x: (x > 500).sum()),
        num_reembolsos_valor_muito_alto = ('Valor total do reembolso', lambda x: (x > 1000).sum())

    ).reset_index()

    #Regra c3 e c4
    df_diario['regra_c3'] = df_diario['num_reembolsos_alto_valor'] >= 3
    df_diario['regra_c4'] = df_diario['num_reembolsos_valor_muito_alto'] >= 2

    #Regra c5
    df_diario['regra_c5'] = df_diario['total_reembolsos'] > 0.7 * df_diario['Valor do contrato']

    #Ordena para poder fazer o calculo de variação
    df_diario = df_diario.sort_values(by=['Nome da transportadora', 'Nome do motorista', 'Data da criação do reembolso Date'])

    #Regras c8 e c9
    df_diario['variacao_num_reembolsos'] = df_diario.groupby(['Nome da transportadora', 'Nome do motorista'])['num_reembolsos'].pct_change() #type: ignore
    df_diario['variacao_total_reembolsos'] = df_diario.groupby(['Nome da transportadora', 'Nome do motorista'])['total_reembolsos'].pct_change() #type: ignore

    # Calcula o valor absoluto da variação e remove NaN
    df_diario['variacao_num_reembolsos'] = abs(df_diario['variacao_num_reembolsos'])
    df_diario['variacao_total_reembolsos'] = abs(df_diario['variacao_total_reembolsos'])

    df_diario = df_diario.dropna(subset=['variacao_num_reembolsos', 'variacao_total_reembolsos'])

    df_diario['regra_c8'] = (df_diario['variacao_num_reembolsos'] >= 0.3) & (df_diario['variacao_total_reembolsos'] >= 0.3)

    #Regra c9
    df_diario['variacao_total_reembolsos_agenciados'] = df_diario['variacao_total_reembolsos'] / df_diario['Número de dias agenciados do contrato']
    df_diario['regra_c9'] = abs(df_diario['variacao_total_reembolsos_agenciados']) >= 0.3


    # Merge com o DataFrame original
    df = pd.merge(df, df_diario[['Data da criação do reembolso Date', 'Nome da transportadora', 'Nome do motorista', 'regra_c3', 'regra_c4', 'regra_c5', 'regra_c8', 'regra_c9', 'variacao_num_reembolsos', 'variacao_total_reembolsos']],
                  on=['Data da criação do reembolso Date', 'Nome da transportadora', 'Nome do motorista'], how='left')

    # Preenche NaN com False (para as linhas que não tem correspondência no df_diario)
    df[['regra_c3', 'regra_c4', 'regra_c5', 'regra_c8', 'regra_c9']] = df[['regra_c3', 'regra_c4', 'regra_c5', 'regra_c8', 'regra_c9']].fillna(False)

    # Define a função de identificação geral (combina individual e agregada)
    def identificar_reembolso_suspeito(row):
        if row['reembolso_suspeito_individual'] or row['regra_c3'] or row['regra_c4'] or row['regra_c5'] or row['regra_c8'] or row['regra_c9']:
            return True
        return False

    # Aplica a função de identificação geral
    df['Suspeito'] = df.apply(lambda row: 'Suspeito' if identificar_reembolso_suspeito(row) else 'Não Suspeito', axis=1)

   #Agrupa para calcular o score
    df_grupo_score = df.groupby(['Nome da transportadora', 'Nome do motorista']).agg(
        dias_suspeitos = ('Suspeito', lambda x: (x == 'Suspeito').sum()),
        total_dias = ('Suspeito', 'count'),
        variacao_media_num_reembolsos = ('variacao_num_reembolsos', 'mean'),
        variacao_media_total_reembolsos = ('variacao_total_reembolsos', 'mean')

    ).reset_index()

    df_grupo_score['proporcao_dias_suspeitos'] = df_grupo_score['dias_suspeitos'] / df_grupo_score['total_dias']

    #Implementação do score composto
    df_grupo_score['score'] = 0.4 * df_grupo_score['proporcao_dias_suspeitos'] + 0.3 * df_grupo_score['variacao_media_num_reembolsos'].fillna(0).abs() + 0.3 * df_grupo_score['variacao_media_total_reembolsos'].fillna(0).abs()

    # Merge com o DataFrame original
    mapeamento_score = df_grupo_score.set_index(['Nome da transportadora', 'Nome do motorista'])['score'].to_dict()

    def obter_score(row):
        transportadora = row['Nome da transportadora']
        motorista = row['Nome do motorista']
        return mapeamento_score.get((transportadora, motorista), 0) # type: ignore

    df['Risco Numérico'] = df.apply(obter_score, axis=1)

    #Remove coluna auxiliar
    df = df.drop('Data da criação do reembolso Date', axis=1, errors='ignore')

    return df


def gerar_relatorio(arquivo_csv, arquivo_saida):
    """Gera um relatório com o 'Risco Numérico'."""
    df = analisar_reembolsos(arquivo_csv)

    if df is None:
        print("Erro ao analisar os reembolsos.")
        return

    # Salva o DataFrame com o 'Risco Numérico' em um arquivo CSV
    df.to_csv(arquivo_saida, index=False)

    print(f"Relatório com o 'Risco Numérico' salvo em: {arquivo_saida}")


if __name__ == "__main__":
    arquivo_csv = "src/data/reembolsos.csv"
    arquivo_saida = "src/data/output/reembolsos_com_score_completo.csv"
    gerar_relatorio(arquivo_csv, arquivo_saida)

/tmp/ipykernel_463861/3962059523.py:111: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[['regra_c3', 'regra_c4', 'regra_c5', 'regra_c8', 'regra_c9']] = df[['regra_c3', 'regra_c4', 'regra_c5', 'regra_c8', 'regra_c9']].fillna(False)


Relatório com o 'Risco Numérico' salvo em: src/data/output/reembolsos_com_score_completo.csv
